In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats

import jb_helper_functions_prep
from jb_helper_functions_prep import create_enc

import prep_telco
from prep_telco import prep_telco_df

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import tree

In [2]:
df = prep_telco_df()
df.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,onlinesecurity_enc,onlinebackup_enc,deviceprotection_enc,techsupport_enc,streamingtv_enc,streamingmovies_enc,contract_enc,paperlessbilling_enc,paymentmethod_enc,churn_enc
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,0,2,0,0,0,0,0,1,2,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,2,0,2,0,0,0,1,0,3,0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,2,2,0,0,0,0,0,1,3,1
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,2,0,2,2,0,0,1,0,0,0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,0,0,0,0,0,0,0,1,2,1


In [3]:
df.internetservice.unique()

array(['DSL', 'Fiber optic', 'No'], dtype=object)

In [4]:
fiber_df = df[df.internetservice == 'Fiber optic']

In [5]:
fiber_train, fiber_test = train_test_split(fiber_df, test_size=.3, random_state=123, stratify=fiber_df[['churn_enc']])

In [6]:
print('Percent of fiber_non-churn: ' + str(fiber_train.churn_enc.value_counts()[0]/fiber_train.churn_enc.count()))
print('Percent of fiber_churn: ' + str(fiber_train.churn_enc.value_counts()[1]/fiber_train.churn_enc.count()))

Percent of fiber_non-churn: 0.5809875403784033
Percent of fiber_churn: 0.41901245962159667


### We see that our train set has 58.1% of customers that did not churn.  This is our benchmark for now.  

In [7]:
fiber_y_train = fiber_train[['churn_enc']]
fiber_y_test = fiber_test[['churn_enc']]

In [8]:
fiber_X_train = fiber_train[['tenure', 'monthlycharges', 'internetservice_enc', 'techsupport_enc', 'contract_enc', 'phoneservice_enc']]
fiber_X_test = fiber_test[['tenure', 'monthlycharges', 'internetservice_enc', 'techsupport_enc', 'contract_enc', 'phoneservice_enc']]

### Let's try a decision tree.

In [12]:
fiber_clf = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=123).fit(fiber_X_train, fiber_y_train)
fiber_y_pred = fiber_clf.predict(fiber_X_train)
fiber_y_pred_proba = fiber_clf.predict_proba(fiber_X_train)
print('Accuracy of Decision Tree classifier on fiber_training set: {:.6f}'
     .format(fiber_clf.score(fiber_X_train, fiber_y_train)))


Accuracy of Decision Tree classifier on fiber_training set: 0.713429


In [13]:
print(classification_report(fiber_y_train, fiber_y_pred))

              precision    recall  f1-score   support

           0       0.75      0.77      0.76      1259
           1       0.66      0.64      0.65       908

   micro avg       0.71      0.71      0.71      2167
   macro avg       0.71      0.70      0.70      2167
weighted avg       0.71      0.71      0.71      2167



In [14]:
fiber_cm = pd.DataFrame(confusion_matrix(fiber_y_train, fiber_y_pred),
             columns=['Pred -', 'Pred +'], index=['Actual -', 'Actual +'])

fiber_cm

,Pred -,Pred +
Actual -,965,294
Actual +,327,581


### Now, let's try logistic regression.

In [15]:
fiber_log_reg = LogisticRegression(random_state=123, solver='saga').fit(fiber_X_train, fiber_y_train)
fiber_y_pred = fiber_log_reg.predict(fiber_X_train)
print('Accuracy of Logistic Regression classifier on training set: {:.6f}'
     .format(fiber_log_reg.score(fiber_X_train, fiber_y_train)))

Accuracy of Logistic Regression classifier on training set: 0.701892


In [16]:
print(classification_report(fiber_y_train, fiber_y_pred))

              precision    recall  f1-score   support

           0       0.76      0.72      0.74      1259
           1       0.63      0.68      0.66       908

   micro avg       0.70      0.70      0.70      2167
   macro avg       0.70      0.70      0.70      2167
weighted avg       0.71      0.70      0.70      2167



In [17]:
fiber_cm = pd.DataFrame(confusion_matrix(fiber_y_train, fiber_y_pred),
             columns=['Pred -', 'Pred +'], index=['Actual -', 'Actual +'])

fiber_cm

,Pred -,Pred +
Actual -,903,356
Actual +,290,618


### Let's see how these compare to a keras ANN model.

In [18]:
from sklearn.preprocessing import StandardScaler

import keras
from keras.models import Sequential
from keras.layers import Dense

sc = StandardScaler()
fiber_X_train = sc.fit_transform(fiber_X_train)
fiber_X_test = sc.transform(fiber_X_test)

Using TensorFlow backend.


In [20]:
cf = Sequential()
cf.add(Dense(output_dim=4, init='uniform', activation='relu', input_dim=6))
cf.add(Dense(output_dim=4, init='uniform', activation='relu'))
cf.add(Dense(output_dim=1, init='uniform', activation='sigmoid'))

cf.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
cf.fit(fiber_X_train, fiber_y_train, nb_epoch=100, batch_size=30)

scores = cf.evaluate(fiber_X_train, fiber_y_train)
print('%s: %.2f%%' % (cf.metrics_names[1], scores[1]*100))

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
2167/2167 [==============================] - 0s 187us/step - loss: 0.6905 - acc: 0.5773
Epoch 2/100
2167/2167 [==============================] - 0s 38us/step - loss: 0.6789 - acc: 0.5810
Epoch 3/100
2167/2167 [==============================] - 0s 38us/step - loss: 0.6509 - acc: 0.6188
Epoch 4/100
2167/2167 [==============================] - 0s 36us/step - loss: 0.6157 - acc: 0.6904
Epoch 5/100
2167/2167 [==============================] - 0s 35us/step - loss: 0.5914 - acc: 0.6904
Epoch 6/100
2167/2167 [==============================] - 0s 36us/step - loss: 0.5797 - acc: 0.6894
Epoch 7/100
2167/2167 [==============================] - 0s 37us/step - loss: 0.5744 - acc: 0.6940
Epoch 8/100
2167/2167 [==============================] - 0s 37us/step - loss: 0.5722 - acc: 0.6945
Epoch 9/100
2167/2167 [==============================] - 0s 36us/step - loss: 0.5700 - acc: 0.6968
Epoch 10/100
2167/2167 [==============================] - 0s

2167/2167 [==============================] - 0s 38us/step - loss: 0.5577 - acc: 0.7107
Epoch 81/100
2167/2167 [==============================] - 0s 38us/step - loss: 0.5578 - acc: 0.7107
Epoch 82/100
2167/2167 [==============================] - 0s 35us/step - loss: 0.5577 - acc: 0.7116
Epoch 83/100
2167/2167 [==============================] - 0s 33us/step - loss: 0.5576 - acc: 0.7093
Epoch 84/100
2167/2167 [==============================] - 0s 36us/step - loss: 0.5573 - acc: 0.7116
Epoch 85/100
2167/2167 [==============================] - 0s 37us/step - loss: 0.5574 - acc: 0.7120
Epoch 86/100
2167/2167 [==============================] - 0s 38us/step - loss: 0.5572 - acc: 0.7093
Epoch 87/100
2167/2167 [==============================] - 0s 37us/step - loss: 0.5572 - acc: 0.7102
Epoch 88/100
2167/2167 [==============================] - 0s 37us/step - loss: 0.5573 - acc: 0.7093
Epoch 89/100
2167/2167 [==============================] - 0s 38us/step - loss: 0.5572 - acc: 0.7102
Epoch 90/100


### Let's run our logistic regression model on the test set.

In [21]:
print('Accuracy of Decision Tree classifier on test set: {:.6f}'
     .format(fiber_clf.score(fiber_X_test, fiber_y_test)))

Accuracy of Decision Tree classifier on test set: 0.645856
